# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This is a classification learning problem because the required output is categorical, in particular a boolean for _passed_ or not. If the output were continuous, i.e. a demical value, regression would apply.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
sd = student_data

# TODO: Calculate number of students
n_students = sd.shape[0]

# TODO: Calculate number of features
n_features = sd.shape[1] - 1  # less one target variable

# TODO: Calculate passing students
n_passed = sd[sd.passed=='yes'].shape[0]

# TODO: Calculate failing students
n_failed = sd[sd.passed=='no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = 1.*n_passed / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' 
    Preprocesses the student data and converts non-numeric
    binary variables into binary (0/1) variables.
    Converts categorical variables into dummy variables.
    '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)

print ( "Processed feature columns ({} total features):\n{}"
        .format(len(X_all.columns), list(X_all.columns)) )

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
SEED = 42

# TODO: Import any additional functionality
from sklearn.cross_validation import train_test_split


# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the 
# number of training and testing points above
X_train, X_test = train_test_split(X_all, train_size=num_train, 
                                   random_state=SEED)
y_train, y_test = train_test_split(y_all, train_size=num_train, 
                                   random_state=SEED)

# Show the results of the split
print ( "Training set has {} samples."
        .format(X_train.shape[0]) )
print ("Testing set has {} samples."
       .format(X_test.shape[0]) )

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will 
1. choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. 
- discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. 
- fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. 
- produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.


**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- ~~Gaussian Naive Bayes (GaussianNB)~~ (features must be assumed independent, unlikely in this case)
- ~~Decision Trees~~ (high variance classifier; since no performance or other constraints so rather use some type ensemble of DTs)
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
    - Random Forest
    - AdaBoost with DTs (not one of three chosen)
    - ~~Bagging with DTs~~ (essentially same as Random Forest)
- K-Nearest Neighbors (KNeighbors) (does not work well with high dimensionality data, but student data seems to have few enough features)
- ~~Stochastic Gradient Descent (SGDC)~~ (not familiar enough with this classifier)
- Support Vector Machines (SVM)
- Logistic Regression (similar to SVM but has advantage over it by giving output with a probablistic interpretation that may be relevant here.)

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

QUESTIONS about how to respond:
- I struggled to find concrete examples of classifiers used in industry (and I searched the web _a lot_). The obvious exception is neural nets which have been in the news with applications by Google, Facebook and others. As such, examples I provided are applications braodly defined in white papers or academic articles.
    - Do you have any ideas where I should look for better, more concrete examples in industry?
    - Tried Kaggle's Kernel section and the code there didn't seem to have relevant examples.
    - I even asked this question on Quora that has yet to be answered: [What are some real-world applications of specific machine learning algorithms?](https://www.quora.com/unanswered/What-are-some-real-world-applications-of-specific-machine-learning-algorithms)

NOTE: Assumptions about data made below are:
- Data is linearly separable.
- Features are _not_ necessarily independent of one another.

**Answer: ** (for enrichment I provide four algorithms)

_1) Random Forest_ (RF; essentially bagging method with DTs)
- Real-world application in industry:
    - Customer churn rate, predicting which customers are going to abandon a product or service (SVM and k-NN as well) 
        - source: [Data Science in Practice](https://www.yhat.com/whitepapers/data-science-in-practice): Five common applications of data science with concrete, real-life use cases Presented by Yhat)
- Strengths of model:
    - Handles categorial (binary) features well
    - Handles large numbers of dimensions and training examples well
    - Maintains low-bias of a single Decision Tree while using an ensemble of DTs' to lower a DTs tendency towards high variance (i.e. overfitting).
    - Data does not have to be linearly separable.
    - Does not expect linear features or even features that interact linearly
    - Works well without much tuning (as opposed to Gradient Boosted Decision Trees and most other classifiers)
    - Non-parametric, so no need to worry about outliers
    - Fast and scalable
- Weaknesses of model:
    - If DTs in the forest become too large, their accuracy can suffer
    - Must rebuild model every time new training data needs to be integrated into the model
    - Others?
- Why it is appropriate for this data?
    - At 48 features, the student data set benefits from an RF's adaptability to high-dimensional training data
    - Works even if features have some dependent relationships between them
    - Generally an algorithm that is very suitable to this type of data, and can be used as a benchmark with which to assess performance of other classifiers.

_2) Logistic Regression_ (LR)
- Real-world application in industry:
    - Financial institutions predicting the likelihood of a homeowner defaulting on a mortgage
        - source: [Wikipedia](https://en.wikipedia.org/wiki/Support_vector_machine#Applications)
- Strengths of model:
    - Fairly robust to noise.
    - Methods available to avoid overfitting.
    - Lots of ways to regularize an LR model.
    - Unlike decision trees or SVMs, output can have useful probabilistic interpretation that can be used for ranking instead of classification.
        - e.g., to easily adjust classification thresholds, to say when you’re unsure, or to get confidence intervals
        - _Question_: would label data need to be converted into binary from 'yes', 'no' categories for this to be effective?
        - easily update your model to take in new data (using an online gradient descent method), unlike DTs or SVMs
- Weaknesses of model:
    - No standardized way for dealing with multi-class problems (fundamentally a binary classifier)
    - Limited expressive power
    - Difficult to run incrementally
- Why it is appropriate for this data?
    - Its probablistic interpretation could be very useful for predicting which students are most likely to fail. This is especially relevant in school districts that have limited resources because they can prioritize certain students over others.

_3) Support Vector Machines_ (SVM)
- Real-world application in industry:
    - Hand-written character recognition 
        - source: [Wikipedia](https://en.wikipedia.org/wiki/Support_vector_machine#Applications)
- Strengths of model:
    - High accuracy model (generally)
    - Memory intensive
    - Hard to interpret
    - When used with a linear kernel, it is very similar to Logistic Regression, but without non-linear kernels SVMs have an advantage over LRs because they can be applied to data that is not necessarily linearly separable.
    - Words well in high-dimensional space.
- Weaknesses of model:
    - Very inefficient in training, so not suitable for most industry scalce applications.
    - No probabilistic interpretation as provided with a Losgistic Regression's output.
    - Can be hard to tune (like finding an appropriate kernel)
    - No standardized way for dealing with multi-class problems (fundamentally a binary classifier)
- Why it is appropriate for this data?
    - Fundamentally a binary classifier
    - Student data is linearly separable

_4) K-Nearest Neighbors_ (k-NN)
- Real-world application in industry:
    - Economic forecasting, predicting financial distress
        - source, PDF: [Application of K-Nearest Neighbor (KNN) Approach for Predicting Economic Events: Theoretical Background](http://www.ijera.com/papers/Vol3_issue5/DI35605610.pdf)
- Strengths of model:
    - Works well with small numbers of dimensions
    - Generally has good perfromance in many contexts.
- Weaknesses of model:
    - Performs poorly with high-dimensional training data (like other instance-based learning techniques)
    - Typically are not useful for understanding the nature of the relationship between the features and class outcome (like other _black box_ models)
    - Uses lots of memory since it must store all instances in training data
- Why it is appropriate for this data?
    - k-NN performs well with low-dimensionality data (if we consider the 48 dimensions in the student data as small).

__Sources:__
Advantages and Disadvantages:
- _Elements of Statistical Learning: Data Mining, Inference, and Prediction_ 2nd Ed. by Trevor Hastie, Robert Tibshirani  & Jerome Friedman (2013)
- [Quora post](https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms): What are the advantages of different classification algorithms?
- [Choosing a Machine Learning Classifier](http://blog.echen.me/2011/04/27/choosing-a-machine-learning-classifier/) by Edwin Chen
- [Chapter 7: Comparison of Classifiers](https://openaccess.leidenuniv.nl/bitstream/handle/1887/13575/Chapter%207.pdf?sequence=12) book unknown (2009)
- [Slides on Random Forest](http://www.dabi.temple.edu/~hbling/8590.002/Montillo_RandomForests_4-2-2009.pdf) from guest lecture at Temple University (4-2-2009)

Applications in industry:
- [Data Science in Practice](https://www.yhat.com/whitepapers/data-science-in-practice): Five common applications of data science with concrete, real-life use cases Presented by Yhat

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ( "Training a {} using a training set size of {}. . ."
            .format(clf.__class__.__name__, len(X_train)) )
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ( "F1 score for training set: {:.4f}."
             .format(predict_labels(clf, X_train, y_train)) )
    print ( "F1 score for test set: {:.4f}.\n"
            .format(predict_labels(clf, X_test, y_test)) )

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = RandomForestClassifier(random_state=SEED)
clf_B = LogisticRegression(random_state=SEED)
clf_C = SVC(random_state=SEED)
clf_D = KNeighborsClassifier()  # extra for enrichment

# TODO: Set up the training set sizes
# NOTE: Unnecessary given looping technique below
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier 
# and each training set size:
# train_predict(clf, X_train, y_train, X_test, y_test)

all_clfs = [clf_A, clf_B, clf_C, clf_D]

for clf in all_clfs:
    print "\n\n***", clf.__class__.__name__, "***"
    for s in range(100, 400, 100):
        train_predict(clf, X_train[:s], y_train[:s], 
                           X_test[:s], y_test[:s])




*** RandomForestClassifier ***
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0340 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.9844.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.7153.

Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0568 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.9964.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7857.

Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0240 seconds
Made predictions in 0.0018 seconds.
F1 score for training set: 0.9951.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.8092.



*** LogisticRegression ***
Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0307 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8593.
Made predictions in 0.0002 seconds.
F1

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

NOTE: All times provided are in seconds.

** Classifer 1 - RandomForestClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0257         |           0.0015       |    0.9844     |       0.7153  |
| 200               |        0.0418          |        0.0019         |      0.9964     |     0.7857     |
| 300               |         0.0228       |       0.0012         |     0.9951     |    0.8092      |

** Classifer 2 - LogisticRegression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0010            |        0.0002       |     0.8593    |     0.7647    |
| 200               |     0.0016             |      0.0003         |    0.8562      |    0.7914      |
| 300               |      0.0030          |      0.0003        |      0.8468      |     0.8060     |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0013         |       0.0011        |     0.8777     |       0.7746   |
| 200               |      0.0026           |        0.0019        |     0.8679     |     0.7815     |
| 300               |       0.0056        |       0.0040         |      0.8761    |     0.7838    |

** Classifer 4 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0006           |         0.0013       |    0.8060    |    0.7246     |
| 200               |     0.0005          |         0.0023     |     0.8800     |    0.7692      |
| 300               |      0.0009         |      0.0057          |     0.8809     |     0.7801    |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Based on tests of four different types of models, the Random Forest classifier is best suited to our purposes. It performs well "out of the box" and thus can be deployed quickly even if the data structure changes in the future. Assuming the size of the training data does not increase drastically in the future, the most costly resource in this project will be the time of the data scientist. Thus, the fact that RFs are easily deployable will save the most money for this project. The longer training time and short predicting time of Random Forests are not relevant to this application.

As for the performance of the model, it will be as good or better than any other for this data, but Random Forests have the potential benefit of indicating relationships between aspects of a student and whether they will succeed academically or not.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Random Forests are a combination of many Decision Trees. A Decision Tree puts together a combination of questions about a student into something like a flowchart. It analyses the data to make sure the questions are asked in the right order so that when we want a prediction about a student, the answers based on details about that student will lead to a prediction about him or her.

Unfortunately, a Decision Tree can start to make irrelevant connections between some questions and answers that work very well for the data we already have, but make poor predictions for new data. A Random Forest solves this by combining lots of DTs, each of which looks at a small slice of the training data. This keeps each tree from becoming too complicated. By combining them together, we can preserve the accuracy of a single DT on the training data, while keeping it from "over-learning" so that it can be applied to new data to make good predictions.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.cross_validation import ShuffleSplit

# TODO: Create the parameters list you wish to tune
parameters = {
              'n_estimators':range(3, 30, 2),
              'min_samples_split':range(1 ,10, 2),
              'criterion':['gini', 'entropy'],
              }

# for Logistic Regression
parameters_LR = {
              'C':np.arange(0.5, 2.0, 0.1),
              'fit_intercept':[True, False],
              'dual':[True, False],
              'intercept_scaling':np.arange(0.5, 15.0, 0.5),
              'tol':np.arange(0.00001, 0.00010, 0.00001),
#               'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag'],
#               'penalty':['l2', 'l1'],
              }

# For Support Vector Machine
parameters_SVC = {
              'C':np.arange(0.5, 5.0, 0.1),
              'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
#               'gamma':[np.arange(0.001,0.005,0.001)],
#               'degree':[1,2,3],
#               'probability':[True, False],
#               'tol':np.arange(0.0001, 0.002, 0.0001),
              }

# TODO: Initialize the classifier
clf = RandomForestClassifier(random_state=SEED)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the 
# f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data 
# and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print ( "Tuned model has a training F1 score of {:.4f}."
       .format(predict_labels(clf, X_train, y_train)) )
print ( "Tuned model has a testing F1 score of {:.4f}."
       .format(predict_labels(clf, X_test, y_test)) )

Made predictions in 0.0021 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0015 seconds.
Tuned model has a testing F1 score of 0.7943.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

|                  | F1 Score (train) | F1 Score (test) |
| :--------------: | :--------------: | :-------------: |
|      Tuned       |       1.0000     |     0.7943      |
|     Untuned      |       0.9444     |     0.7832      |

Oddly enough, the tuned model performs worse. After looking at [this post](https://discussions.udacity.com/t/gridsearch-leading-to-worse-f1-score/164148) in the discussion forums for this project, I learned that this anomaly is due to chance. The data set is small, and the validation method in the grid search indicated that particular parameter set performs better than the default parameter set, but happens to perfrom slightly worse on the test set.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.